In [1]:
#==============================================================================
# CELL 1: IMPORT LIBRARIES
#==============================================================================
"""
Import semua libraries yang dibutuhkan untuk hyperparameter tuning.
"""
# Data manipulation
import pandas as pd
import numpy as np
from collections import Counter

# Machine Learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import (
    RandomizedSearchCV, 
    cross_val_score, 
    StratifiedKFold,
    cross_validate
)

# Imbalanced learning
from imblearn.over_sampling import SMOTE, ADASYN

# Evaluation metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, auc,
    confusion_matrix, classification_report,
    precision_recall_curve, average_precision_score,
    cohen_kappa_score, matthews_corrcoef,
    balanced_accuracy_score, make_scorer
)

# Statistical testing
from scipy import stats
from statsmodels.stats.contingency_tables import mcnemar

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Utilities
import warnings
import json
import os
import joblib
from datetime import datetime
import time

warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Plot settings
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['savefig.dpi'] = 300
sns.set_style("whitegrid")
sns.set_palette("husl")

print("="*80)
print("FASE 4: HYPERPARAMETER TUNING & MODEL OPTIMIZATION")
print("="*80)
print("\n✅ All libraries imported successfully!")
print("="*80)

FASE 4: HYPERPARAMETER TUNING & MODEL OPTIMIZATION

✅ All libraries imported successfully!


In [2]:
#==============================================================================
# CELL 2: LOAD DATA & BASELINE MODEL
#==============================================================================
"""
Load preprocessed data dan baseline model untuk comparison.
"""
print("\n" + "="*80)
print("📂 LOADING DATA & BASELINE MODEL")
print("="*80)

# Load training data
X_train = pd.read_csv('../data/processed/X_train.csv')
y_train = pd.read_csv('../data/processed/y_train.csv').values.ravel()

# Load test data
X_test = pd.read_csv('../data/processed/X_test.csv')
y_test = pd.read_csv('../data/processed/y_test.csv').values.ravel()

# Load baseline model
baseline_model = joblib.load('../models/baseline_rf_model.pkl')

# Load baseline metrics
with open('../results/metrics/03_baseline_metrics.json', 'r') as f:
    baseline_metrics = json.load(f)

print("\n✅ Data Loaded:")
print(f"  • Training samples: {len(X_train)}")
print(f"  • Test samples: {len(X_test)}")
print(f"  • Features: {X_train.shape[1]}")
print(f"  • Feature names: {X_train.columns.tolist()}")

print("\n✅ Class Distribution:")
train_dist = Counter(y_train)
test_dist = Counter(y_test)
print(f"  • Training: Class 0={train_dist[0]}, Class 1={train_dist[1]} (ratio: {train_dist[0]/train_dist[1]:.2f})")
print(f"  • Test: Class 0={test_dist[0]}, Class 1={test_dist[1]} (ratio: {test_dist[0]/test_dist[1]:.2f})")

print("\n✅ Baseline Model Performance (Recap):")
print(f"  • Test Accuracy: {baseline_metrics['test_metrics']['accuracy']:.4f}")
print(f"  • Test ROC-AUC: {baseline_metrics['test_metrics']['roc_auc']:.4f}")
print(f"  • Test F1-Score: {baseline_metrics['test_metrics']['f1_score']:.4f}")
print(f"  • Training Time: {baseline_metrics['model_info']['training_time_seconds']:.2f}s")

# Get baseline predictions for later comparison
y_test_pred_baseline = baseline_model.predict(X_test)

print("\n" + "="*80)


📂 LOADING DATA & BASELINE MODEL

✅ Data Loaded:
  • Training samples: 1508
  • Test samples: 377
  • Features: 24
  • Feature names: ['Age_Encoded', 'Gender_Encoded', 'Education_Encoded', 'country_Australia', 'country_Canada', 'country_New Zealand', 'country_Other', 'country_Republic of Ireland', 'country_UK', 'country_USA', 'ethnicity_Asian', 'ethnicity_Black', 'ethnicity_Mixed-Black/Asian', 'ethnicity_Mixed-White/Asian', 'ethnicity_Mixed-White/Black', 'ethnicity_Other', 'ethnicity_White', 'Nscore', 'Escore', 'Oscore', 'Ascore', 'Cscore', 'Impulsive', 'SS']

✅ Class Distribution:
  • Training: Class 0=568, Class 1=940 (ratio: 0.60)
  • Test: Class 0=142, Class 1=235 (ratio: 0.60)

✅ Baseline Model Performance (Recap):
  • Test Accuracy: 0.8647
  • Test ROC-AUC: 0.9271
  • Test F1-Score: 0.8898
  • Training Time: 0.47s



In [3]:
#==============================================================================
# CELL 3: DEFINE HYPERPARAMETER SEARCH SPACE
#==============================================================================
"""
Define comprehensive hyperparameter search space untuk RandomizedSearchCV.
"""
print("\n" + "="*80)
print("🔧 DEFINING HYPERPARAMETER SEARCH SPACE")
print("="*80)

# Hyperparameter distributions
param_distributions = {
    # Number of trees
    'n_estimators': [100, 200, 300, 500, 1000],
    
    # Tree depth (add constraint to reduce overfitting)
    'max_depth': [10, 20, 30, 40, None],
    
    # Minimum samples to split a node (increase to reduce overfitting)
    'min_samples_split': [2, 5, 10, 20],
    
    # Minimum samples at leaf node
    'min_samples_leaf': [1, 2, 4, 8],
    
    # Number of features for best split
    'max_features': ['sqrt', 'log2', 0.3, 0.5],
    
    # Bootstrap sampling
    'bootstrap': [True, False],
    
    # Class weights (for handling imbalance)
    'class_weight': ['balanced', 'balanced_subsample', None],
    
    # Split criterion
    'criterion': ['gini', 'entropy']
}

print("\n✅ Search Space Defined:")
for param, values in param_distributions.items():
    print(f"  • {param:20s}: {values}")

# Calculate total combinations
total_combinations = 1
for values in param_distributions.values():
    total_combinations *= len(values)

print(f"\n✅ Total possible combinations: {total_combinations:,}")
print(f"✅ RandomizedSearchCV will sample: 100 combinations")
print(f"✅ With 5-fold CV: 100 × 5 = 500 model fits")
print(f"✅ Estimated time: 10-30 minutes (depending on hardware)")

print("\n" + "="*80)


🔧 DEFINING HYPERPARAMETER SEARCH SPACE

✅ Search Space Defined:
  • n_estimators        : [100, 200, 300, 500, 1000]
  • max_depth           : [10, 20, 30, 40, None]
  • min_samples_split   : [2, 5, 10, 20]
  • min_samples_leaf    : [1, 2, 4, 8]
  • max_features        : ['sqrt', 'log2', 0.3, 0.5]
  • bootstrap           : [True, False]
  • class_weight        : ['balanced', 'balanced_subsample', None]
  • criterion           : ['gini', 'entropy']

✅ Total possible combinations: 19,200
✅ RandomizedSearchCV will sample: 100 combinations
✅ With 5-fold CV: 100 × 5 = 500 model fits
✅ Estimated time: 10-30 minutes (depending on hardware)



In [4]:
#==============================================================================
# CELL 4: RANDOMIZED SEARCH CV
#==============================================================================
"""
Perform RandomizedSearchCV untuk find optimal hyperparameters.
PALING LAMA: 10-30 menit tergantung hardware!
"""
print("\n" + "="*80)
print("🔍 HYPERPARAMETER TUNING - RANDOMIZED SEARCH CV")
print("="*80)

# Define cross-validation strategy
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize base Random Forest
rf_base = RandomForestClassifier(
    random_state=42,
    n_jobs=-1,  # Use all CPU cores
    verbose=0
)

# Initialize RandomizedSearchCV
print("\n⚙️  Setting up RandomizedSearchCV...")
random_search = RandomizedSearchCV(
    estimator=rf_base,
    param_distributions=param_distributions,
    n_iter=100,  # Sample 100 combinations
    scoring='roc_auc',  # Optimize for ROC-AUC as per proposal
    cv=cv_strategy,
    verbose=2,  # Show progress
    random_state=42,
    n_jobs=-1,  # Parallel processing
    return_train_score=True
)

print("\n✅ RandomizedSearchCV Configuration:")
print(f"  • Estimator: RandomForestClassifier")
print(f"  • n_iter: 100 (sample 100 combinations)")
print(f"  • Scoring: roc_auc")
print(f"  • CV: 5-fold Stratified")
print(f"  • n_jobs: -1 (all cores)")
print(f"  • Total fits: 100 × 5 = 500")

# Perform hyperparameter search
print("\n🚀 Starting hyperparameter search...")
print("⏱️  This will take 10-30 minutes depending on your hardware...")
print("-"*80)

start_time = time.time()
random_search.fit(X_train, y_train)
tuning_time = time.time() - start_time

print("\n" + "="*80)
print(f"✅ HYPERPARAMETER TUNING COMPLETED!")
print(f"⏱️  Total time: {tuning_time/60:.2f} minutes ({tuning_time:.0f} seconds)")
print("="*80)


🔍 HYPERPARAMETER TUNING - RANDOMIZED SEARCH CV

⚙️  Setting up RandomizedSearchCV...

✅ RandomizedSearchCV Configuration:
  • Estimator: RandomForestClassifier
  • n_iter: 100 (sample 100 combinations)
  • Scoring: roc_auc
  • CV: 5-fold Stratified
  • n_jobs: -1 (all cores)
  • Total fits: 100 × 5 = 500

🚀 Starting hyperparameter search...
⏱️  This will take 10-30 minutes depending on your hardware...
--------------------------------------------------------------------------------
Fitting 5 folds for each of 100 candidates, totalling 500 fits

✅ HYPERPARAMETER TUNING COMPLETED!
⏱️  Total time: 5.03 minutes (302 seconds)


In [5]:
#==============================================================================
# CELL 5: BEST HYPERPARAMETERS
#==============================================================================
"""
Extract dan display best hyperparameters found.
"""
print("\n" + "="*80)
print("🏆 BEST HYPERPARAMETERS FOUND")
print("="*80)

best_params = random_search.best_params_
best_score = random_search.best_score_

print("\n✅ Best Parameters:")
print("-"*80)
for param, value in sorted(best_params.items()):
    print(f"  • {param:20s}: {value}")

print(f"\n✅ Best Cross-Validation Score:")
print(f"  • ROC-AUC: {best_score:.4f}")

# Get best model
best_rf_model = random_search.best_estimator_

print(f"\n✅ Best Model Info:")
print(f"  • n_estimators: {best_rf_model.n_estimators}")
print(f"  • max_depth: {best_rf_model.max_depth}")
print(f"  • min_samples_split: {best_rf_model.min_samples_split}")
print(f"  • min_samples_leaf: {best_rf_model.min_samples_leaf}")
print(f"  • max_features: {best_rf_model.max_features}")
print(f"  • criterion: {best_rf_model.criterion}")
print(f"  • bootstrap: {best_rf_model.bootstrap}")
print(f"  • class_weight: {best_rf_model.class_weight}")

print("\n" + "="*80)


🏆 BEST HYPERPARAMETERS FOUND

✅ Best Parameters:
--------------------------------------------------------------------------------
  • bootstrap           : True
  • class_weight        : None
  • criterion           : gini
  • max_depth           : 20
  • max_features        : log2
  • min_samples_leaf    : 8
  • min_samples_split   : 10
  • n_estimators        : 500

✅ Best Cross-Validation Score:
  • ROC-AUC: 0.8984

✅ Best Model Info:
  • n_estimators: 500
  • max_depth: 20
  • min_samples_split: 10
  • min_samples_leaf: 8
  • max_features: log2
  • criterion: gini
  • bootstrap: True
  • class_weight: None



In [6]:
#==============================================================================
# CELL 6: EVALUATE TUNED MODEL ON TEST SET
#==============================================================================
"""
Evaluate tuned model pada test set dengan comprehensive metrics.
"""
print("\n" + "="*80)
print("📊 EVALUATING TUNED MODEL ON TEST SET")
print("="*80)

# Predictions
print("\n⚙️  Generating predictions...")
y_train_pred_tuned = best_rf_model.predict(X_train)
y_train_proba_tuned = best_rf_model.predict_proba(X_train)[:, 1]

y_test_pred_tuned = best_rf_model.predict(X_test)
y_test_proba_tuned = best_rf_model.predict_proba(X_test)[:, 1]

print("✅ Predictions generated!")

# Calculate comprehensive metrics
def calculate_all_metrics(y_true, y_pred, y_pred_proba, set_name=""):
    """Calculate all evaluation metrics"""
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    metrics = {
        'Set': set_name,
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred, zero_division=0),
        'Recall': recall_score(y_true, y_pred, zero_division=0),
        'Specificity': tn / (tn + fp) if (tn + fp) > 0 else 0,
        'F1-Score': f1_score(y_true, y_pred, zero_division=0),
        'Balanced Accuracy': balanced_accuracy_score(y_true, y_pred),
        'ROC-AUC': roc_auc_score(y_true, y_pred_proba),
        'PR-AUC': average_precision_score(y_true, y_pred_proba),
        'Cohen Kappa': cohen_kappa_score(y_true, y_pred),
        'MCC': matthews_corrcoef(y_true, y_pred),
        'TP': int(tp),
        'TN': int(tn),
        'FP': int(fp),
        'FN': int(fn)
    }
    
    return metrics

# Calculate metrics
train_metrics_tuned = calculate_all_metrics(y_train, y_train_pred_tuned, 
                                            y_train_proba_tuned, "Training")
test_metrics_tuned = calculate_all_metrics(y_test, y_test_pred_tuned, 
                                           y_test_proba_tuned, "Test")

# Display results
tuned_df = pd.DataFrame([train_metrics_tuned, test_metrics_tuned]).set_index('Set')

print("\n✅ TUNED MODEL PERFORMANCE:")
print("="*80)
display_cols = ['Accuracy', 'Precision', 'Recall', 'Specificity', 'F1-Score', 
                'Balanced Accuracy', 'ROC-AUC', 'PR-AUC', 'Cohen Kappa', 'MCC']
print(tuned_df[display_cols].round(4).to_string())

print("\n✅ Confusion Matrix Breakdown:")
print("-"*80)
cm_cols = ['TP', 'TN', 'FP', 'FN']
print(tuned_df[cm_cols].to_string())

# Check overfitting
train_test_gap = train_metrics_tuned['Accuracy'] - test_metrics_tuned['Accuracy']
print(f"\n✅ Overfitting Analysis:")
print(f"  • Training Accuracy: {train_metrics_tuned['Accuracy']:.4f}")
print(f"  • Test Accuracy: {test_metrics_tuned['Accuracy']:.4f}")
print(f"  • Gap: {train_test_gap:.4f} ({train_test_gap*100:.2f}%)")

if train_test_gap > 0.10:
    print("  • Status: ⚠️  Still some overfitting (gap > 10%)")
elif train_test_gap > 0.05:
    print("  • Status: ⚠️  Moderate overfitting (gap 5-10%)")
else:
    print("  • Status: ✅ Good generalization (gap < 5%)")

print("\n" + "="*80)


📊 EVALUATING TUNED MODEL ON TEST SET

⚙️  Generating predictions...
✅ Predictions generated!

✅ TUNED MODEL PERFORMANCE:
          Accuracy  Precision  Recall  Specificity  F1-Score  Balanced Accuracy  ROC-AUC  PR-AUC  Cohen Kappa    MCC
Set                                                                                                                 
Training    0.8707     0.9045  0.8862       0.8451    0.8952             0.8656   0.9441  0.9680       0.7264 0.7267
Test        0.8621     0.9031  0.8723       0.8451    0.8874             0.8587   0.9347  0.9612       0.7095 0.7102

✅ Confusion Matrix Breakdown:
--------------------------------------------------------------------------------
           TP   TN  FP   FN
Set                        
Training  833  480  88  107
Test      205  120  22   30

✅ Overfitting Analysis:
  • Training Accuracy: 0.8707
  • Test Accuracy: 0.8621
  • Gap: 0.0086 (0.86%)
  • Status: ✅ Good generalization (gap < 5%)

